In [1]:
import pandas as pd
from pathlib import Path

In [2]:
#Apertura de los datasets en formato DataFrame
DATA_DIR = Path("../datasets")

df_ventas = pd.read_csv(DATA_DIR / 'ventas.csv')
df_clasificados = pd.read_csv(DATA_DIR / 'productos_clasificados.csv')


df_ventas.head(3)

,fecha,product_name,category_off,ventas,precio
0,2022-12-01,Néctar de durazno 1 L,juice-box,10,33.72
1,2022-12-01,Arroz blanco 500 g,rice-white-dry,3,45.84
2,2022-12-01,Betabel,fruits-vegetables,33,30.22


In [3]:
#Del DataFrame donde estan los productos clasificados solo se usaran las siguientes dos columnas
df_clasificados = df_clasificados[['product_name', 'perecedero']]
df_clasificados.head(3)

,product_name,perecedero
0,Néctar de durazno 1 L,0
1,Sardinas en salsa 155 g,0
2,Queso gouda 300 g,1


In [4]:
#Se hace una copia del DataFrame de las ventas
df_ventas_cp = df_ventas.copy()

In [5]:
#Se Hace la union del DataFrame de perecederos, es decir a cada producto se le asigna si es perecedero o no
df_ventas_perecederos = pd.merge(df_ventas_cp, df_clasificados, how="left", on= "product_name")
df_ventas_perecederos.loc[df_ventas_perecederos['category_off'] == 'fruits-vegetables', 'perecedero'] = 1
df_ventas_perecederos[df_ventas_perecederos['perecedero'].isna()]

,fecha,product_name,category_off,ventas,precio,perecedero


In [6]:
#Se crean las siguientes tres columnas con valores por defecto
df_ventas_perecederos['en_temporada'] = 0
df_ventas_perecederos['temp_inicio_mes'] = 1 
df_ventas_perecederos['temp_fin_mes'] = 12

In [7]:
#Se abre el DataFrame con las temporadas de todas las frutas y verduras
df_frutas_verduras_temp = pd.read_csv(DATA_DIR / 'frutas-verduras_temporada_normalizado.csv')
df_frutas_verduras_temp.head(3)

,Unnamed: 0,nombre,tipo,temporada_pico,todo_el_anio,temporada_pico_inicio,temporada_pico_fin
0,0,Aguacate,fruits-vegetables,Dic-May (alta); disponible todo el año,True,12.0,5.0
1,1,Ciruela,fruits-vegetables,Jul–Ago,False,7.0,8.0
2,2,Durazno,fruits-vegetables,Jul–Sep,False,7.0,9.0


In [8]:
#Pequenia funcion que se encarga de acer una normalizacion simple de los nombres de los productos
#Solo quita acentos 
import unicodedata

def normalizar(texto):
    """
    La funcion realiza lo siguiente:
    - Convierte el texto a minúsculas.
    - Elimina espacios en blanco al inicio y al final.
    - Quita acentos y caracteres diacríticos (normalización Unicode NFD).
    - Convierte valores NaN en una cadena vacía.

    Parámetros
    ----------
    texto : str o cualquier tipo convertible a string
        Texto original a normalizar.
    """
    
    if pd.isna(texto):
        return ""
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize("NFD", texto)
    texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")  # quita acentos
    return texto

# Crear columnas normalizadas
df_frutas_verduras_temp['nombre'] = df_frutas_verduras_temp['nombre'].apply(normalizar)
df_ventas_perecederos['product_name'] = df_ventas_perecederos['product_name'].apply(normalizar)


In [9]:
#De todas las frutas y verduras las busca en el dataFrame de ventas para poder asignar el inicio y final de la temporada
for _, row in df_frutas_verduras_temp.iterrows():
    nombre = row['nombre'] 

    # Buscar los product_name que contengan ese nombre
    mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)

    # Asignar inicio y fin de temporada
    df_ventas_perecederos.loc[mask, 'temp_inicio_mes'] = row['temporada_pico_inicio']
    df_ventas_perecederos.loc[mask, 'temp_fin_mes']    = row['temporada_pico_fin']
    

/tmp/ipykernel_115730/2108136344.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)
/tmp/ipykernel_115730/2108136344.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)
/tmp/ipykernel_115730/2108136344.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)
/tmp/ipykernel_115730/2108136344.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)


In [10]:
#Este bloque  se encarga de calcular si un producto está en temporada según el mes de la venta y los meses de inicio y fin de su temporada. 

df_ventas_perecederos['fecha'] = pd.to_datetime(df_ventas_perecederos['fecha'])
mes = df_ventas_perecederos['fecha'].dt.month
ini = df_ventas_perecederos['temp_inicio_mes']
fin = df_ventas_perecederos['temp_fin_mes']

#temporada normal (inicio <= fin)
caso_1 = (ini <= fin) & (mes >= ini) & (mes <= fin)

#temporada que cruza año (inicio > fin)
caso_2 = (ini > fin) & ((mes >= ini) | (mes <= fin))

df_ventas_perecederos['en_temporada'] = 0
df_ventas_perecederos.loc[caso_1 | caso_2, 'en_temporada'] = 1


In [11]:
'''
Debido a que hay productos disponibles todo el año (del mes 1 al 12),
la temporada de inicio y la temporada de fin aparecen con valores nulos.
Para garantizar que estos productos sean considerados como disponibles
durante cualquier mes, se asignan valores por defecto:
- temp_inicio_mes = 1  (enero)
- temp_fin_mes   = 12 (diciembre)
'''
df_ventas_perecederos['temp_inicio_mes'] = df_ventas_perecederos['temp_inicio_mes'].fillna(1)
df_ventas_perecederos['temp_fin_mes'] = df_ventas_perecederos['temp_fin_mes'].fillna(12)


In [12]:
#Este bloque  se encarga de calcular si un producto está en temporada según el mes de la venta y los meses de inicio y fin de su temporada. 

df_ventas_perecederos['fecha'] = pd.to_datetime(df_ventas_perecederos['fecha'])
mes = df_ventas_perecederos['fecha'].dt.month
ini = df_ventas_perecederos['temp_inicio_mes']
fin = df_ventas_perecederos['temp_fin_mes']

#temporada normal (inicio <= fin)
caso_1 = (ini <= fin) & (mes >= ini) & (mes <= fin)

#temporada que cruza año (inicio > fin)
caso_2 = (ini > fin) & ((mes >= ini) | (mes <= fin))

df_ventas_perecederos['en_temporada'] = 0
df_ventas_perecederos.loc[caso_1 | caso_2, 'en_temporada'] = 1


In [13]:
df_ventas_perecederos.to_csv(DATA_DIR/'ventas_normalizado.csv')
df_ventas_perecederos

,fecha,product_name,category_off,ventas,precio,perecedero,en_temporada,temp_inicio_mes,temp_fin_mes
0,2022-12-01,nectar de durazno 1 l,juice-box,10,33.72,0.0,0,7.0,9.0
1,2022-12-01,nectar de durazno 1 l,juice-box,10,33.72,0.0,0,7.0,9.0
2,2022-12-01,arroz blanco 500 g,rice-white-dry,3,45.84,0.0,1,1.0,12.0
3,2022-12-01,arroz blanco 500 g,rice-white-dry,3,45.84,0.0,1,1.0,12.0
4,2022-12-01,arroz blanco 500 g,rice-white-dry,3,45.84,0.0,1,1.0,12.0
...,...,...,...,...,...,...,...,...,...
840627,2025-11-30,cafe molido 500 g,coffee-ground,1,54.91,0.0,1,1.0,12.0
840628,2025-11-30,cafe molido 500 g,coffee-ground,1,54.91,0.0,1,1.0,12.0
840629,2025-11-30,queso fresco 250 g,cheese-fresh,17,55.21,1.0,1,1.0,12.0
840630,2025-11-30,queso fresco 250 g,cheese-fresh,17,55.21,1.0,1,1.0,12.0
